In [1]:
import torch
import torchvision
import cv2 
import numpy as np
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import torch.nn.functional as F
import time
from encoder import Encoder
from maneuver import Maneuver



def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.75:
        robot.set_motors(.5,.5)
        time.sleep(.5)
        encoder.get_data()
    else:
        move.execute_maneuver()
        encoder.get_data()



In [2]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model.load_state_dict(torch.load('best_model.pth'))

device = torch.device('cuda')
model = model.to(device)


mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

goal_distance = float(input("Enter a goal distance in cm: "))
robot = Robot()
encoder = Encoder(goal_distance,robot)
move = Maneuver(robot)
        
update({'new': camera.value})  

camera.observe(update, names='value') 

Enter a goal distance in cm:  60


0.05  and  0.0
0.0  and  0.05
5.1  and  5.94
0.0  and  0.05
0.0  and  0.0
0.05  and  0.0
0.05  and  0.05
0.05  and  0.05
0.05  and  0.0


In [10]:
from jetbot import Robot
robot = Robot()
robot.stop()